# Анализ данных о бронировании отелей

In [1]:
import pandas as pd

## Считывание и подготовка данных

In [2]:
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', sep=';')

In [3]:
bookings_head = bookings.head(7)

In [4]:
bookings_head

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


### Узнаем размер таблицы

In [5]:
bookings.shape

(119390, 21)

### Узнаем тип переменных в таблице

In [6]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

### Привидем названия столбцов к однородному виду

In [7]:
bookings.columns = bookings.columns.str.replace(' ', '_').str.lower()

In [8]:
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

## Работа с данными

### Узнаем клиенты каких стран совершили наибольшее число успешных бронирований, возьмем топ-5 стран

In [9]:
bookings \
.query('is_canceled == 0') \
.country \
.value_counts() \
.head(5)


PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

### Найдем на сколько ночей в среднем бронируют отели типа "City Hotel" и "Resort Hotel"

In [10]:
bookings \
.groupby(['hotel'], as_index=False) \
.aggregate({'stays_total_nights':'mean'}) \
.round(decimals=2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


### Найдем колличество случаев, когда тип номера клиента отличается от изначально забронированного 

In [11]:
bookings \
    .query('assigned_room_type != reserved_room_type') \
    .shape[0]

14917

### Узнаем на какой месяц чаще всего оформляли бронь в 2016 и 2017 году

#### 2016 год

In [12]:
bookings \
    .query('arrival_date_year == 2016') \
    .value_counts('arrival_date_month') \
    .idxmax()

'October'

#### 2017 год

In [13]:
bookings \
    .query('arrival_date_year == 2017') \
    .value_counts('arrival_date_month') \
    .idxmax()

'May'

### Сгруппируем данные по годам и узнаем на какой месяц бронирования отеля типа City Hotel отменялись чаще всего

In [14]:
bookings \
    .query('hotel == "City Hotel" and is_canceled == 1') \
    .groupby('arrival_date_year', as_index=False) \
    .arrival_date_month \
    .value_counts()
    

,arrival_date_year,arrival_date_month,count
0,2015,September,1543
1,2015,October,1321
2,2015,August,1232
3,2015,July,939
4,2015,December,668
5,2015,November,301
6,2016,October,1947
7,2016,June,1720
8,2016,September,1567
9,2016,April,1539


### Проверим какая из числовых характеристик имеет наибольшее занчение

In [15]:
bookings[['adults', 'children', 'babies']] \
    .mean() \
    .round(decimals=2)

adults      1.86
children    0.10
babies      0.01
dtype: float64

### Создадим колонку "total_kids", объединив столбцы children и babies, найдем среднее значение детей для отелей

In [16]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

In [17]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0


In [18]:
bookings \
    .groupby('hotel', as_index=False) \
    .aggregate({'total_kids':'mean'}, ) \
    .sort_values('total_kids', ascending=False) \
    .round(decimals=2)

,hotel,total_kids
1,Resort Hotel,0.14
0,City Hotel,0.10


### Создадим переменную "has_kids", которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка, в противном случае False

In [19]:
bookings['has_kids'] = bookings['total_kids'] > 0

In [20]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids,has_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0,False
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0,False


### Найдем сколько клиентов было потеряно в процессе бронирования, процент оттока плиентов

In [21]:
churn_rate = (
    bookings.query('is_canceled == 1').shape[0] 
    / bookings.shape[0]
)
churn_rate = round(churn_rate * 100, 2)

In [22]:
churn_rate

37.04

### Проверим среди какой группы пользователей с детьми или без детей показатель оттока выше

#### Найдем ооток пользователей без детей

In [23]:
no_kids_churn_rate = (
    bookings.query('is_canceled == 1 and has_kids == False').shape[0] 
    / bookings.query('has_kids == False').shape[0]
)
no_kids_churn_rate = round(no_kids_churn_rate * 100, 2)

In [24]:
no_kids_churn_rate

37.22

#### Найдем оттот среди пользователей с детьми

In [25]:
kids_churn_rate = (
    bookings.query('is_canceled == 1 and has_kids == True').shape[0] 
    / bookings.query('has_kids == True').shape[0]
)
kids_churn_rate = round(kids_churn_rate * 100, 2)

In [26]:
kids_churn_rate

34.92